<a href="https://colab.research.google.com/github/Benjamin025/Benjamin025/blob/main/radar_based_flood_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mounnt Google Drive
import os                                     # data access
import google.colab                           # Google Colab
import time                                   # time assessment
import sys
if not os.path.isdir('/content/drive'):
    google.colab.drive.mount('/content/drive')

try:
    import snappy                             # SNAP Python interface
    import jpy                                # Python-Java bridge
except:
    with google.colab.output.use_tags('snappy'):
        sys.stdout.write('\nPreparing conda environment...\n')
        sys.stdout.flush()
        !pip install -q condacolab &> /dev/null
        import condacolab
        condacolab.install_miniconda()
        sys.stdout.write('\nInstalling snappy package...\n')
        sys.stdout.flush()
        !conda install -c terradue -c conda-forge snap=8.0.0 &> /dev/null
    google.colab.output.clear(output_tags='snappy')
try:
    import geopandas                          # data analysis and manipulation
except:
    with google.colab.output.use_tags('geopandas'):
        sys.stdout.write('\nInstalling geopandas package...\n')
        sys.stdout.flush()
        !pip install geopandas &> /dev/null
    google.colab.output.clear(output_tags='geopandas')
try:
    from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt  # interface to Open Access Hub
except:
    with google.colab.output.use_tags('sentinelsat'):
        sys.stdout.write('\nInstalling sentinelsat package...\n')
        sys.stdout.flush()
        !pip install sentinelsat &> /dev/null
    google.colab.output.clear(output_tags='sentinelsat')
try:
    from ipyfilechooser import FileChooser    # file chooser widget
except:
    with google.colab.output.use_tags('ipyfilechooser'):
        sys.stdout.write('\nInstalling ipyfilechooser package...\n')
        sys.stdout.flush()
        !pip install ipyfilechooser &> /dev/null
    google.colab.output.clear(output_tags='ipyfilechooser')
import ipywidgets
if int(ipywidgets.__version__.split('.')[0]) >= 8:
    !pip install "ipywidgets>=7,<8"  &> /dev/null

# status update
with google.colab.output.use_tags('initialization'):
    sys.stdout.write('\nInitialization successful!')
    sys.stdout.flush()
time.sleep(1)
google.colab.output.clear(output_tags='initialization')

Mounted at /content/drive


In [ ]:
##specifying the polarization yo be used
##VH is the dual polarization \i.e vertical transmit and horizontal receive##
from typing_extensions import TypeVarTuple
# polarisations to be processed
Polarisation  = 'VH'                    #@param ["VH", "VV", "both"]

DownloadImage = TypeVarTuple                 #@param {type:"boolean"}

# show intermediate results if set to 'True'
PlotResults   = True                 #@param {type:"boolean"}

In [ ]:
##installing jpy module
!pip install jpy --upgrade


/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [ ]:
import geopandas
!pip install gdal-utils

/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Using cached GDAL-3.8.5.tar.gz (802 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!apt-get install openjdk-11-jre-headless -qq > /dev/null

/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [ ]:
##importing the necessary modules
# MODULE                                      # DESCRIPTION
import sys
!pip install snappy
import snappy # SNAP Python interface
!apt-get install openjdk-11-jre-headless -qq > /dev/null

!pip install geopandas                        # Python-Java bridge
import geopandas
!pip install  sentinelsat.sentinel
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt  # interface to Open Access Hub

from ipyfilechooser import FileChooser    # file chooser widget

import os                                     # data access
import google.colab                           # Google Colab
import matplotlib.pyplot as plt               # create visualizations
import numpy as np                            # scientific comupting
import json                                   # JSON encoder and decoder
import glob                                   # data access
import ipywidgets                             # interactive UI controls
import time                                   # time assessment
import shutil                                 # file operations
import folium                                 # visualization
from folium import plugins                    # visualization
from folium.plugins import MiniMap, Draw, Search # visualization
import skimage.filters                        # threshold calculation
import functools                              # higher-order functions and operations
from datetime import date                     # dates, times and intervalls
from IPython.display import display           # visualization
!pip install gdal-utils
from zipfile import ZipFile                   # file management

/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
ERROR: Could not find a version that satisfies the requirement sentinelsat.sentinel (from versions: none)
ERROR: No matching distribution found for sentinelsat.sentinel
/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Using cached gdal_utils-3.8.5.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached GDAL-3.8.5.tar.gz (802 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.

In [ ]:
##function dfinitions
def getAOI(path):
    try:
        file = readJSONFromAOI(path)
    except:
        print('No AOI file was found. Please draw and download the area of interest by clicking the Export-button')
        print('inside the map or directly upload a locally stored AOI file using the dialogue below the map.\n')
        # create map
        f = folium.Figure(height=500)
        m = folium.Map(location=[0, 0], zoom_start=2.5, control_scale=True).add_to(f)
        # add custom basemap
        basemaps['Google Satellite Hybrid'].add_to(m)
        # add a layer control panel to the map
        m.add_child(folium.LayerControl())
        # add minimap
        m.add_child(MiniMap(tile_layer=basemaps['Google Satellite'], position='bottomright'))
        # add draw control
        draw = Draw(export=True, filename='AOI_manual_%s.geojson' % str(date.today()), draw_options={'polyline': False, 'circle': False, 'marker': False, 'circlemarker': False})
        draw.add_to(m)
        # display map
        updater = display(f, display_id='m')
        print('\n')
        # open upload section
        os.chdir('/content')
        uploaded = google.colab.files.upload()
        for fn in uploaded.keys():
            # copy uploaded file to GDrive folder
            aoi_path = os.path.join(directory, 'AOI')
            if not os.path.isdir(aoi_path):
                os.mkdir(aoi_path)
            shutil.copy2('/content/%s' % fn, aoi_path)
            # remove original file
            os.remove('/content/%s' % fn)
            file_path = '%s/%s' % (aoi_path, fn)
        file = readJSONFromAOI(aoi_path)

    return file


In [ ]:
# Function looks for AOI file, converts to GeoJSON if not given and returns path to GeoJSON
def readJSONFromAOI(path):
    # check for GeoJSON file in 'AOI' subfolder
    if len(glob.glob('%s/*.geojson' % path)) == 1:
        file = glob.glob('%s/*.geojson' % path)[0]
    elif len(glob.glob('%s/*.json' % path)) == 1:
        file = glob.glob('%s/*.json' % path)[0]

In [ ]:
##plotting the dataset

# plot band and histogram of 'Band'-type input and threshold
# SNAP API: https://step.esa.int/docs/v6.0/apidoc/engine/
def plotBand(band, threshold, binary=False):
    # color stretch
    vmin, vmax = 0, 1
    # read pixel values
    w = band.getRasterWidth()
    h = band.getRasterHeight()
    band_data = np.zeros(w * h, np.float32)
    band.readPixels(0, 0, w, h, band_data)
    band_data.shape = h, w
    # color stretch
    if binary:
        cmap = plt.get_cmap('binary')
    else:
        vmin = np.percentile(band_data, 2.5)
        vmax = np.percentile(band_data, 97.5)
        cmap = plt.get_cmap('gray')
    # plot band
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,6))
    ax1.imshow(band_data, cmap=cmap, vmin=vmin, vmax=vmax)
    ax1.set_title(band.getName())
    # plot histogram
    band_data.shape = h * w
    ax2.hist(np.asarray(band_data[band_data != 0], dtype='float'), bins=2048)
    ax2.axvline(x=threshold, color='r')
    ax2.set_title('Histogram: %s' % band.getName())

    for ax in fig.get_axes():
        ax.label_outer()

In [ ]:
# set working directory
directory = '/content/drive/MyDrive/radar-based-flood-mapping'
if not os.path.isdir(directory):
    os.mkdir(directory)

# Add custom basemap to folium
basemaps = {
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True,
        show = False
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True,
        show = False
    )
}

In [ ]:
##user input bassed on the copernicus open access hub
Username      = 'ndungubenjamin025@gmail.com'              #@param {type:"string"}
Password      = 'Naivasha2384#'              #@param {type:"string"}
SensingPeriod_Start  = '2023-12-25'     #@param {type:"date"}
SensingPeriod_Stop   = '2024-05-05'     #@param {type:"date"}


In [ ]:
# search for and display available Sentinel-1 tiles
def queri(footprint):
    # print status
    with google.colab.output.use_tags('loading'):
        sys.stdout.write('\nLoading...')
        sys.stdout.flush()
    # search Copernicus Open Access Hub for products with regard to input footprint and sensing period
    period = (date(int(SensingPeriod_Start.split('-')[0]), int(SensingPeriod_Start.split('-')[1]), int(SensingPeriod_Start.split('-')[2])),
              date(int(SensingPeriod_Stop.split('-')[0]), int(SensingPeriod_Stop.split('-')[1]), int(SensingPeriod_Stop.split('-')[2])))
    try:
        products = api.query(footprint, date=period, platformname='Sentinel-1', producttype='GRD')
        print('Successfully connected to Copernicus Open Access Hub.\n', flush=True)
    except:
        sys.exit('\nLogin data not valid. Please change username and/or password.')

         # convert to GeoJSON for plot
    products_json = api.to_geojson(products)
    # raise warning that no image is available in given sensing period
    if not products_json['features']:
        sys.exit('\nNo Sentinel-1 images available. Please change sensing period in user input section.')

         # convert to GeoJSON for plot
    products_json = api.to_geojson(products)
    # raise warning that no image is available in given sensing period
    if not products_json['features']:
        sys.exit('\nNo Sentinel-1 images available. Please change sensing period in user input section.')

        # adds index to dataframe
    indices = []
    for i in range (1, len(products_df.index)+1):
        indices.append('Tile %d' % i)
        products_json.features[i-1].properties['index'] = ' Tile %d' % i
    products_df.insert(0, 'index', indices, True)


    # load products in folium framework for visualization
    s1_tiles = folium.GeoJson(
        products_json,
        name='S1 tiles',
        show=True,
        style_function=lambda feature: {'fillColor': 'royalblue', 'fillOpacity' : 0.2},
        highlight_function=lambda x: {'fillOpacity' : 0.4},
        tooltip=folium.features.GeoJsonTooltip(
            fields=['index', 'beginposition'],
            aliases=['Index:','Date:'],
        ),
    ).add_to(m)

       # add custom basemap
    basemaps['Google Satellite Hybrid'].add_to(m)
    # add a layer control panel to the map
    m.add_child(folium.LayerControl())
    # clear status update
    google.colab.output.clear(output_tags='loading')
    # update map
    updater.update(m)

     # print table with download buttons
    grid = ipywidgets.GridspecLayout(len(products_df.index)+1, 5)
    grid[0,0] = ipywidgets.HTML('<h3>Index</h3>')
    grid[0,1] = ipywidgets.HTML('<h3>Date</h3>')
    grid[0,2] = ipywidgets.HTML('<h3>Polarisation</h3>')
    grid[0,3] = ipywidgets.HTML('<h3>Size</h3>')
    for i in range(len(products_df.index)):
        grid[i+1,0] = ipywidgets.Label(products_df['index'][i])
        grid[i+1,1] = ipywidgets.Label(str(products_df['beginposition'][i]))
        grid[i+1,2] = ipywidgets.Label(products_df['polarisationmode'][i])
        grid[i+1,3] = ipywidgets.Label(products_df['size'][i])
        grid[i+1,4] = ipywidgets.Button(description = 'Download')
        grid[i+1,4].on_click(functools.partial(on_downloadButton_clicked, tile_id=products_df.values[i][-1]))
    display(grid)


# download chosen Sentinel-1 tile in subfolder 'input'
def on_downloadButton_clicked(b, tile_id):
    # get product information
    product_info = api.get_product_odata(tile_id)
    # check whether product is available
    if product_info['Online']:
        # check if input folder exists, if not create input folder
        input_path = os.path.join(directory, 'input')
        if not os.path.isdir(input_path):
            os.mkdir(input_path)
        # change into 'input' subfolder for storing product
        os.chdir(input_path)
        # status update
        print('\nProduct %s is online. Starting download.' % tile_id, flush=True)
        # download product
        api.download(tile_id)
        # change back to previous working directory
        os.chdir(directory)
    # error message when product is not available
    else:
        print('\nProduct %s is not online. Must be requested manually.\n' % tile_id, flush=True)



# check user input whether image download is requested
if 'DownloadImage' in globals():
 if DownloadImage:
    # connect to the API
    api = SentinelAPI(Username, Password, 'https://scihub.copernicus.eu/dhus')
    # get path to AOI
    file = getAOI('%s/AOI' % directory)
    # open AOI GeoJSON file and store data
    with open(file, 'r') as f:
        data_json = json.load(f)
    # define map center depending on GeoJSON internal structure
    try:
        # GeoJSON format if KMZ is given
        center = [data_json['features'][0]['geometry']['coordinates'][0][0][0][1],
                  data_json['features'][0]['geometry']['coordinates'][0][0][0][0]]
    except:
        # GeoJSON format if JSON or SHP is given
        center = [data_json['features'][0]['geometry']['coordinates'][0][0][1],
                  data_json['features'][0]['geometry']['coordinates'][0][0][0]]
    # create map
    f = folium.Figure(height=500)
    m = folium.Map(location=center, zoom_start=6, control_scale=True).add_to(f)
    # add AOI to map
    folium.GeoJson(file, name='AOI', style_function = lambda x: {'color':'green'}).add_to(m)
    footprint = geojson_to_wkt(data_json)
    updater = display(f, display_id='m')

    # search for available Sentinel-1 tiles
    queri(footprint)


In [ ]:
##processing of the sar products
def applySubset(S1A_IW_SLC__1SDV_20240102T154907_20240102T154934_051929_064638_F150):
    # set correct path of input file and create S1 product
    if len(files) == 1:
        file_path = S1A_IW_SLC__1SDV_20240102T154907_20240102T154934_051929_064638_F150
    else:
        file_path = S1A_IW_SLC__1SDV_20240102T154907_20240102T154934_051929_064638_F150.selected
    S1_source = snappy.ProductIO.readProduct(file_path)

 # read geographic coordinates from Sentinel-1 image meta data
    meta_data = S1_source.getMetadataRoot().getElement('Abstracted_Metadata')
    print(meta_data)

     # refines center of map according to Sentinel-1 image
    S1_center = (meta_data.getAttributeDouble('centre_lat'), meta_data.getAttributeDouble('centre_lon'))

     # defines polygon illustrating Sentinel-1 image
    polygon_geom = {
      "type": "FeatureCollection",
      "features":
                [{"type": "Feature",
                "properties": {},
                "geometry": {"type": "Polygon", "coordinates": [[[meta_data.getAttributeDouble('first_near_long'), meta_data.getAttributeDouble('first_near_lat')],
                                                                [meta_data.getAttributeDouble('last_near_long'), meta_data.getAttributeDouble('last_near_lat')],
                                                                [meta_data.getAttributeDouble('last_far_long'), meta_data.getAttributeDouble('last_far_lat')],
                                                                [meta_data.getAttributeDouble('first_far_long'), meta_data.getAttributeDouble('first_far_lat')],
                                                                [meta_data.getAttributeDouble('first_near_long'), meta_data.getAttributeDouble('first_near_lat')]]]}}]}

   # get path to AOI
    file = getAOI('%s/AOI' % directory)

    # open GeoJSON file and store data
    with open(file, 'r') as f:
        data_json = json.load(f)
    footprint = geojson_to_wkt(data_json)

      # creates map
    f = folium.Figure(height=500)
    m = folium.Map(location = S1_center, zoom_start = 7.5, control_scale=True).add_to(f)

     # add S1 tile to map
    folium.GeoJson(polygon_geom, name='Sentinel-1 tile').add_to(m)
    # add AOI to map
    folium.GeoJson(file, name='AOI', style_function = lambda x: {'color':'green'}).add_to(m)
    # add custom basemap
    basemaps['Google Satellite Hybrid'].add_to(m)
    # add a layer control panel to the map
    m.add_child(folium.LayerControl())
    # display map
    updater = display(f, display_id='m')

    # Subset operator
    parameters = snappy.HashMap()
    parameters.put('copyMetadata', True)
    geom = snappy.WKTReader().read(footprint)
    parameters.put('geoRegion', geom)
    parameters.put('sourceBands', sourceBands)
    S1_crop = snappy.GPF.createProduct('Subset', parameters, S1_source)
    # status update
    print('\nSubset successfully generated.\n', flush=True)

    # run processing process
    processing(S1_crop)


# calculate and return threshold of 'Band'-type input
# SNAP API: https://step.esa.int/docs/v6.0/apidoc/engine/
def getThreshold(S1_band):
    # read band
    w = S1_band.getRasterWidth()
    h = S1_band.getRasterHeight()
    band_data = np.zeros(w * h, np.float32)
    S1_band.readPixels(0, 0, w, h, band_data)
    band_data.shape = h * w
    # calculate threshold using Otsu method
    threshold_otsu = skimage.filters.threshold_otsu(band_data)
    # calculate threshold using minimum method
    threshold_minimum = skimage.filters.threshold_minimum(band_data)
    # get number of pixels for both thresholds
    numPixOtsu = len(band_data[abs(band_data - threshold_otsu) < 0.1])
    numPixMinimum = len(band_data[abs(band_data - threshold_minimum) < 0.1])

    # if number of pixels at minimum threshold is less than 0.1% of number of pixels at Otsu threshold
    if abs(numPixMinimum/numPixOtsu) < 0.001:
        # adjust band data according
        if threshold_otsu < threshold_minimum:
            band_data = band_data[band_data < threshold_minimum]
            threshold_minimum = skimage.filters.threshold_minimum(band_data)
        else:
            band_data = band_data[band_data > threshold_minimum]
            threshold_minimum = skimage.filters.threshold_minimum(band_data)
        numPixMinimum = len(band_data[abs(band_data - threshold_minimum) < 0.1])
    # check for final threshold
    if abs(numPixMinimum/numPixOtsu) < 0.001:
        threshold = threshold_otsu
    else:
        threshold = threshold_minimum

    return threshold

# calculate binary mask of 'Product'-type intput with respect expression in string array
def binarization(S1_product, expressions):

    BandDescriptor = jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')
    targetBands = jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', len(expressions))

    # loop through bands
    for i in range(len(expressions)):
        targetBand = BandDescriptor()
        targetBand.name = '%s' % S1_product.getBandNames()[i]
        targetBand.type = 'float32'
        targetBand.expression = expressions[i]
        targetBands[i] = targetBand

    parameters = snappy.HashMap()
    parameters.put('targetBands', targetBands)
    mask = snappy.GPF.createProduct('BandMaths', parameters, S1_product)

    return mask

###processing steps
# processing steps
def processing(S1_crop):
    # Apply-Orbit-File operator
    print('1. Apply Orbit File:          ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    # continue with calculation in case no orbit file is available yet
    parameters.put('continueOnFail', True)
    S1_Orb = snappy.GPF.createProduct('Apply-Orbit-File', parameters, S1_crop)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # ThermalNoiseRemoval operator
    print('2. Thermal Noise Removal:     ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('removeThermalNoise', True)
    S1_Thm = snappy.GPF.createProduct('ThermalNoiseRemoval', parameters, S1_Orb)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Calibration operator
    print('3. Radiometric Calibration:   ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('outputSigmaBand', True)
    S1_Cal = snappy.GPF.createProduct('Calibration', parameters, S1_Thm)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Speckle-Filter operator
    print('4. Speckle Filtering:         ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('filter', 'Lee')
    parameters.put('filterSizeX', 5)
    parameters.put('filterSizeY', 5)
    S1_Spk = snappy.GPF.createProduct('Speckle-Filter', parameters, S1_Cal)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Conversion from linear to db operator
    S1_Spk_db = snappy.GPF.createProduct('LinearToFromdB', snappy.HashMap(), S1_Spk)


  # Terrain-Correction operator
    print('5. Terrain Correction:        ', end='', flush=True)
    parameters = snappy.HashMap()
    parameters.put('demName', 'SRTM 1Sec HGT')
    parameters.put('demResamplingMethod', 'BILINEAR_INTERPOLATION')
    parameters.put('imgResamplingMethod', 'NEAREST_NEIGHBOUR')
    parameters.put('pixelSpacingInMeter', 10.0)
    parameters.put('nodataValueAtSea', False)
    parameters.put('saveSelectedSourceBand', True)
    S1_TC = snappy.GPF.createProduct('Terrain-Correction', parameters, S1_Spk_db)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)




    # Binarization
    print('6. Binarization:              ', end='', flush=True)
    start_time = time.time()
    # add GlobCover band
    parameters = snappy.HashMap()
    parameters.put('landCoverNames', 'GlobCover')
    GlobCover = snappy.GPF.createProduct('AddLandCover', parameters, S1_TC)
    # empty string array for binarization band maths expression(s)
    expressions = ['' for i in range(S1_TC.getNumBands())]
    # empty array for threshold(s)
    thresholds = np.zeros(S1_TC.getNumBands())
    # loop through bands
    for i in range(S1_TC.getNumBands()):
        # calculate threshold of band and store in float array
        # use S1_Spk_db product for performance reasons. S1_TC causes 0-values
        # which distort histogram and thus threshold result
        thresholds[i] = getThreshold(S1_Spk_db.getBandAt(i))
        # formulate expression according to threshold and store in string array
        expressions[i] = 'if (%s < %s && land_cover_GlobCover != 210) then 1 else NaN' % (S1_TC.getBandNames()[i], thresholds[i])
    # do binarization
    S1_floodMask = binarization(GlobCover, expressions)
    print('--- %.2f seconds ---' % (time.time() - start_time), flush=True)


 # Speckle-Filter operator
    print('7. Speckle Filtering:         ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('filter', 'Median')
    parameters.put('filterSizeX', 5)
    parameters.put('filterSizeY', 5)
    # define flood mask as global for later access
    global S1_floodMask_Spk
    S1_floodMask_Spk = snappy.GPF.createProduct('Speckle-Filter', parameters, S1_floodMask)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # output
    if PlotResults:
        print('8. Plot:                      ', end='', flush=True)
        start_time = time.time()
        for i in range(S1_TC.getNumBands()):
            plotBand(S1_TC.getBandAt(i), thresholds[i])
        print('--- %.2f seconds ---' % (time.time() - start_time), flush=True)


#####CODE#####
#############3
##############
#############
# filter required polarisation(s) and set output file name accordingly
Polarisation = 'both' # or 'VH' or 'VV'
if Polarisation == 'both':
    sourceBands = 'Amplitude_VH,Intensity_VH,Amplitude_VV,Intensity_VV'
    output_extensions   = 'processed_VHVV'
elif Polarisation == 'VH':
    sourceBands = 'Amplitude_VH,Intensity_VH'
    output_extensions   = 'processed_VH'
elif Polarisation == 'VV':
    sourceBands = 'Amplitude_VV,Intensity_VV'
    output_extensions   = 'processed_VV'


####################
######################
# path of Sentinel-1 .zip input file
input_path = os.path.join(directory, 'input/before/S1A_IW_SLC__1SDV_20240102T154907_20240102T154934_051929_064638_F150')
# empty string array to store Sentinel-1 files in 'input' subfolder
files = []
# add files to list
for file in glob.glob1(input_path, '*.zip'):
    files.append(file)
# select input file and start processing if there is only one available Sentinel-1 file
if len(files) == 1:
    input_name = files[0]
    print('Selected:  %s\n' % input_name, flush=True)
    # apply subset according to JSON data
    applySubset('%s/%s' % (input_path, input_name))
# open dialogue to select input file if more or less than one is available
else:
    print('More or less than one Sentinel-1 files have been found. Please select.', flush=True)
    fc = FileChooser(input_path)
    fc.filter_pattern = '*.zip'
    fc.register_callback(applySubset)
    display(fc)